# 通过对留出集的时序评估进行个性化

本笔记本在很大程度上遵循了基本笔记本内容，并进行了额外的调整，为每个用户保留 10% 的"未来"数据。然后，我们设置了一个推理端点，以提供推荐并在外部对留出数据进行评估。

另一个小的差异是我们预测的是观看次数而不是评分。我们发现与推荐每个人都会喜欢的流行电影（无个性化需求）相比，展示冷门（但高度个性化）电影的推荐更有趣。

In [ ]:
import boto3, os
import json
import numpy as np
import pandas as pd
import time
from botocore.exceptions import ClientError
!pip install tqdm
from tqdm import tqdm_notebook
from metrics import mean_reciprocal_rank, ndcg_at_k, precision_at_k

In [ ]:
suffix = str(np.random.uniform())[4:9]

In [ ]:
bucket = "demo-temporal-holdout-"+   suffix        # replace with the name of your S3 bucket
filename = "DEMO-temporal-holdout.csv"

In [ ]:
!aws s3 mb s3://{bucket}

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# 下载并处理基准数据

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip
data = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['USER_ID','ITEM_ID','EVENT_VALUE', 'TIMESTAMP'])

In [ ]:
pd.set_option('display.max_rows', 5)

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

### 抽取每位用户最后 10% 的交互作为留出测试 

In [ ]:
ranks = data.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')
data = data.join((ranks>0.9).to_frame('holdout'))
holdout = data[data['holdout']].drop('holdout', axis=1)
data = data[~data['holdout']].drop('holdout', axis=1)

In [ ]:
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

### 上载数据

In [ ]:
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

# 创建架构

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-temporal-schema-"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## 数据集和数据集组

### 创建数据集组

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-temporal-dataset-group-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

### 创建“交互”数据集类型

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "DEMO-temporal-dataset-"+suffix
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## S3 存储桶访问 Personalize 的权限

### 将策略附加到 S3 存储桶

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

### 创建具有相应权限的角色

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

# 创建数据集导入作业

这是您的交互数据上载

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-temporal-dataset-import-job-"+suffix,
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

### 等待数据集导入作业和数据集导入作业运行处于 ACTIVE 状态

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

# 创建解决方案

In [ ]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

对于不同的场景，有很多配方。在此示例中，我们只有交互数据，所以我们将从基本配方中选择一个。

| 可行吗？ | 配方 | 说明 
|-------- | -------- |:------------
| 是 | aws-popularity-count | 根据用户-项目交互数据集中针对该项的事件计数，计算项目的人气。
| 是 | aws-hrnn | 预测用户将与之交互的项目。一种分层递归神经网络，可以对用户-项目交互的时序进行建模。
| N – 需要元数据 | aws-hrnn-metadata | 预测用户将与之交互的项目。HRNN 具有来自上下文（用户-项目交互元数据）、用户元数据（用户数据集）和项目元数据（项目数据集）的附加功能
| N - 用于 Bandit 算法，需要元数据 | aws-hrnn-coldstart | 预测用户将与之交互的项目。HRNN 元数据，可对新项目进行个性化探索。
| N – 用于基于项目的查询 | aws-sims | 根据用户-项目交互数据集中相同用户历史记录中项目的共同存在情况，计算与给定项目相似的项目
| N – 用于重新排列简短列表 | aws-personalized-ranking | 为用户重新排列项目列表。训练用户-项目交互数据集。


我们（或 autoML）可以运行所有这些基本配方，并从内部指标中选择性能最佳的模型。我们建议进行比较，尤其是与人气基准进行比较，以通过个性化查看指标的提升情况。但是，在此演示中，我们将选择配方 aws-hrnn 并专注于外部评估。

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

### 创建并等待您的解决方案
这个过程分为两步
1. 创建解决方案
2. 创建解决方案版本

In [ ]:
create_solution_response = personalize.create_solution(
    name = "DEMO-temporal-solution-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

### 等待解决方案版本状态变为 ACTIVE

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 获取解决方案的指标

In [ ]:
get_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_metrics_response, indent=2))

# 创建并等待活动

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-temporal-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

### 等待活动状态变为 ACTIVE

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

# 使用外部指标进行评估

评估指标的说明请参考：https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html

例如，假设我们推荐了四个项目，其中两个是相关的，$r=[0,1,0,1]$。在本用例中，指标是：

|名称	|示例	|说明
|:------|:----------|:----------
|Precision@K	|$\frac{2}{4} = 0.5$	|相关项总数除以推荐项总数。
|平均倒数排名（MRR@K）	|${\rm mean}(\frac{1}{2} + \frac{1}{4}) = 0.375$	|通过计算所有相关项的反向位置的平均值来考虑位置效应。
|归一化折损累计增益（NDCG@K）	|$\frac{\frac{1}{\log(1 + 2)} + \frac{1}{\log(1 + 4)}}{\frac{1}{\log(1 + 1)} + \frac{1}{\log(1 + 2)}} = 0.65$	|通过根据相关项目的位置应用反对数权重来考虑位置效应，并通过理想推荐的最大可能分数进行归一化。
|平均精度（AP@K）	|${\rm mean}(\frac{1}{2} + \frac{2}{4}) = 0.5$	|Average precision@K，其中 K 是每个相关项目的位置。

这些指标在两个方面不同于内部指标：
* 它们的评估时间不同，这可能会导致点击率的不同。我们建议始终将评估保持在相同的时间段内，以避免时间漂移。
* 示例外部评估可能会留出并考虑多个项作为基准真相，而内部评估只留出每个用户历史记录中的最后一项作为基准真相。对于应该留出多少项并没有绝对的偏好；我们建议设计与实际用例相似的评估方法。

In [ ]:
relevance = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))

### 可选：删除之前的购买历史记录后，结果稍好

In [ ]:
rel_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    topk = [int(x['itemId']) for x in rec_response['itemList']]
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    

    rel_dedup.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_dedup]))

### 尝试与作为虚拟推荐者的人气基线进行比较

In [ ]:
topk = data.groupby("ITEM_ID").TIMESTAMP.count().sort_values(ascending=False).iloc[:100].index.values

In [ ]:
rel_popular = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_items = topk[:25]
    rel_popular.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_popular]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_popular]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_popular]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_popular]))

### 人气基线重复删除用户历史记录

In [ ]:
rel_pop_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    
    rel_pop_dedup.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_pop_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_pop_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_pop_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_pop_dedup]))

# 清理

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)

personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

# 如果您使用的是个人存储桶，请谨慎执行此单元格！

In [ ]:
!aws s3 rm s3://{bucket} --recursive